This notebook demonstrates how to use GPT to translate natural language questions into KQL queries.

In [123]:
import os
from dotenv import load_dotenv
from typing import Annotated
from openai import AzureOpenAI
import sqlite3
from typing import Any, List
import pandas as pd
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
from azure.identity import DefaultAzureCredential

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY= os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME")
AZURE_OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT_NAME")

KUSTO_URI = os.getenv("NL_TO_KQL_KUSTO_URI")
KUSTO_DATABASE = os.getenv("NL_TO_KQL_KUSTO_DATABASE")
KUSTO_TABLE = os.getenv("NL_TO_KQL_KUSTO_TABLE")

try:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    print(ex)

In [124]:
llm = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)

def call_openAI(user_prompt, use_json_object=True):
    system_message = """You are an assistant designed to answer questions."""
    
    if use_json_object:
        response = llm.chat.completions.create(
            model=AZURE_OPENAI_GPT4_DEPLOYMENT_NAME,
            messages = [
                {"role":"system","content":system_message},
                {"role":"user","content":user_prompt}
                ],
            response_format={ "type": "json_object" }
        )
        return response.choices[0].message.content
    else:
        response = llm.chat.completions.create(
            model=AZURE_OPENAI_GPT4_DEPLOYMENT_NAME,
            messages = [
                {"role":"system","content":system_message},
                {"role":"user","content":user_prompt}
                ]
        )
    return response.choices[0].message.content


In [125]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

kcsb = KustoConnectionStringBuilder.with_az_cli_authentication(KUSTO_URI)
print(kcsb)
client = KustoClient(kcsb)
kusto_db = KUSTO_DATABASE

Data Source=https://trd-m78emrpuh6devrz4ty.z4.kusto.fabric.microsoft.com;Initial Catalog=NetDefaultDB;AAD Federated Security=True;Authority Id=organizations;AZ CLI=True


In [126]:
def execute_query(kusto_query: str):
    response = client.execute(kusto_db, kusto_query)
    df = dataframe_from_result_table(response.primary_results[0])
    df
    return df


def get_table_creation_kql() -> str:
    table_schema = ".create-merge table Stocks (Date:datetime, Open:real, High:real, Low:real, Close:real, AdjClose:real, Volume:string, Ticker:string) "
    return table_schema

In [127]:
from string import Template

def load_template(filename: str) -> Template:
    file = open(filename, "r")
    content = file.read()
    file.close()
    template = Template(content)
    return template

In [128]:
from typing import List, Optional, Tuple
import pandas as pd
import json

def get_correction_prompt(schema: str,question: str,query: str,sql_error_message: str) -> str:
    tbl_creation_kql = get_table_creation_kql()
    template_file_name = "./prompts/correction_prompt.txt"
    template = load_template(template_file_name)
    prompt = template.substitute(schema_prompt=get_schema_prompt(schema),question = question, query = query, sql_error_message = sql_error_message)
    return prompt

def extract_json(response: str):
    json_response = None
    if '{' in response and '}' in response:
        response = response.replace('\n', '')
        json_response = json.loads(response)
    return json_response["explanation"], json_response["query"]
    
def get_schema_prompt() -> str:
    tbl_creation_kql = get_table_creation_kql()
    template_file_name = "./prompts/schema_prompt.txt"
    template = load_template(template_file_name)
    prompt = template.substitute(tbl_creation_kql=tbl_creation_kql)
    return prompt

def get_prompt(question: str) -> str:
    schema_prompt = get_schema_prompt()
    template_file_name = "./prompts/user_prompt.txt"
    template = load_template(template_file_name)
    prompt = template.substitute(schema_prompt=schema_prompt, question=question)
    return prompt

def get_final_answer_prompt(question: str,query: str,explanation: str,result: pd.DataFrame) -> str:
    schema_prompt = get_schema_prompt()
    template_file_name = "./prompts/final_answer_prompt.txt"
    template = load_template(template_file_name)
    prompt = template.substitute(schema_prompt=schema_prompt, question=question, query=query, explanation=explanation, result=result.to_string())
    return prompt

def respond(question: str, chat_history: List[Tuple[str, str]]) -> Tuple:
    user_prompt = get_prompt(question)
    ua_response = call_openAI(user_prompt, True)
    explanation, query = extract_json(ua_response)

    if query is None:
        return '', chat_history, None, '', explanation

    success = False
    for _ in range(5):
        try:
            query_result = execute_query(query)
            success = True
            break
        except Exception as kql_error_message:
            kql_error_prompt = get_correction_prompt(question, query, str(kql_error_message))
            response = call_openAI(kql_error_prompt, True)
            explanation, query = extract_json(response)
            if query is None:
                return '', chat_history, None, '', explanation

    if success:
        final_answer_prompt = get_final_answer_prompt(question, query, explanation, query_result)
        chat_response = call_openAI(final_answer_prompt, False)
        chat_history.append((question, chat_response))
    else:
        query_result = ''

    return '', chat_history, query_result, query, explanation

In [122]:
respond("What is the average price for each stock symbol in the February 2013?", [])

('',
 [('What is the average price for each stock symbol in the February 2013?',
   "The KQL query provided summarizes the average 'Close' price for each stock symbol ('Ticker') in the Stocks table during the month of February in the year 2013. The filtered records range from February 1, 2013, to February 28, 2013, and the results are grouped by the stock ticker. The output displays each ticker with its corresponding average 'Close' price during this period. For instance, ticker 'AAPL' (Apple Inc.) had an average 'Close' price of 16.314605, while ticker 'MSFT' (Microsoft Corporation) had an average of 27.704211. This information can be used to analyze the stock performance of different companies during that month.")],
     Ticker   AvgPrice
 0      OKE  41.215475
 1        J  48.548421
 2       HD  66.738947
 3      HAS   39.79421
 4      PFE  25.849895
 ..     ...        ...
 462    FLT  65.527368
 463    MPC  40.260789
 464    XYL  27.725263
 465   META  28.087895
 466    ZTS  32.554